# A Gentle Introduction to Python for High Performance Computing (HPC)
![Python Logo](https://www.python.org/static/img/python-logo.png)
## Introduction
Python is a very popular language.  It is taught in schools and universities, and has support on many platforms.  It is not hard to see why:  The language is dynamically typed, so you don't have to worry about whether your datatype is an integer or a string.  It hides many of the more complicated problems that cause headaches in many other languages, such as pointers/references and memory allocation.  There is also a large number of modules included within the base distribution, allowing complicated programs to be created in very few lines.  All these things lead to a faster development rate than might be seen in many other languages, and higher programmer productivity.

Unfortunately, this comes at a cost.  Python is not natively high performing.  Memory use can easily balloon and is difficult to manage, since it is abstracted away and hidden from the programmer.

## Why is Python slow?
The reference Python interpreter, CPython, is the most commonly used Python interpreter.  This interpreter has been improved over the years, but is far from perfect.  Python is still much slower and less energy (and memory) efficient than the compiled languages commonly used in HPC and scientific computing (C, C++ and Fortran).  Memory management is more difficult too, as you do not have explicit control over how the interpreter uses memory and memory use is not necessarily obvious to the programmer.

## Multi-threading
Multithreading is also problematic for CPython, as it relies on the [global interpreter lock (GIL)](https://docs.python.org/3/glossary.html#term-global-interpreter-lock), a mutex, to ensure thread safety.  This means that the interpreter can only execute one thread at a time.  This is true no matter how many CPUs are available to the interpreter: Even with many CPUs, it will execute only one instruction will execute at a time.

What does this mean for multiple threads?  It means that they will each take turns to execute.  While one thread is executing, the others will have to wait.  The interpreter will switch between threads either when it reaches an I/O operation or every 100 instructions.  Context switching also incurs overheads, as the data and instructions required by each thread must be switched in and out of the CPU's registers and caches.

All this means that using [threading](https://docs.python.org/3/library/threading.html) is generally slower than just writing a single-threaded program.  The best thing to do in Python is to avoid using [threading](https://docs.python.org/3/library/threading.html) altogether.

### Common quick fixes

None of these are recommended, but they are commonly used.  They provide replacements for CPython, but cannot fix all the problems inherent in the language.

* [Numba](https://numba.pydata.org/)
    * A JIT compiler for a subset of Python and NumPy

* [Pypy](https://www.pypy.org/)
    * Another JIT compiler
    * Poor support for Python extensions

* [Cython](https://cython.org/)
    * Translates your Python to C, so you need familiarity with compiling and linking C programs
    * Dependency hell..

## What can we do?

* Write efficient code
    * Profile and fix your software
    * Make use of [SWIG]() to call more efficient C or C++ libraries where there are hot spots, if appropriate
* Use tools like:
    * [NumPy](https://numpy.org/doc/stable/)
    * [Multiprocessing](https://docs.python.org/3/library/multiprocessing.html) (now part of the standard library!)
    * In the HPC world, consider using [MPI](https://www.mpi-forum.org/docs/) to scale up via [MPI4Py](https://mpi4py.readthedocs.io/en/stable/)

## Writing Efficient Python Code

This is not specific to Python, but it may have a much larger impact than in many other languages.  When you are programming, always think about how it will be executed.  Will unnecessary work be done by the computer to achieve the result?  Can you write something that is more efficient?

Which one is faster?

`def search_a(haystack, needle):
        for item in haystack:
                if item == needle:
                        return True
        return False
`


`def search_b(haystack, needle):
        return_value = False
        for item in haystack:
                if item == needle:
                        return_value = True
        return return_value
`

Take a few minutes to read through and compare each one.

<div class="alert alert-block alert-info">
    <b>Hint:</b> What happens if the first item you look at is the needle and the haystack contains 100 items? 
</div>

<div class="alert alert-block alert-success">
    <b>Answer:</b> This is a fairly simple example.  If you read through carefully, you can see that the first version will return immediately upon finding the needle.  The second one will find the needle, but then continue comparing through the rest of the items in the haystack.  This is clearly sub-optimal if you know that there is only one needle.
</div>

Let's try another example.

Which one is faster?

`def search_c(haystack, needle):
    return any((item == needle for item in haystack))
`

`def search_d(haystack, needle):
    return any([item == needle for item in haystack])
`

This is much more Pythonic, and although they look the same at first glance, a careful read will show that they are not identical.

<div class="alert alert-block alert-info">
<b>Did you spot the difference?</b>  A simple change of parenthesis for brackets is all that is needed to tell the interpreter to treat these very differently.
</div>

What does this small difference mean in practice?

## Profiling

Fortunately, profiling tools are part of the standard Python library.  You should always profile before trying to improve the runtime of your code, as otherwise you could waste time optimising an area of your code that does not need it and find you have made no difference to the overall runtime.

As we're currently unsure what the difference is between our two examples above, let's use the profiling tools to examine how they behave with a simple testcase.

### timeit
First, we will use [timeit](https://docs.python.org/3/library/timeit.html) to see how long each one takes to execute.

In [ ]:
# First, we'll need to import it so that we can use it
import timeit
# For this example, we'll use random to generate the data
import random

# Generate some sample data
setup = 'haystack = random.sample(range(1,1000), 512); needle = random.choice(haystack)'

Now we can try timing some examples by inserting calls to timeit functions in the right places and printing the results.  Note that our decision to use random data may affect the results, so always consider the impact of your data and its characteristics as well.

In [ ]:
# Here are our two examples
def search_c(haystack, needle):
    return any((item == needle for item in haystack))

def search_d(haystack, needle):
    return any([item == needle for item in haystack])

Having each one defined as a separate function makes it very easy for us to time them.  We need only call them within the `timeit.timeit()` wrapper function.

In [ ]:
print("Using a generator expression takes: ",
    timeit.timeit('search_c(haystack, needle)', setup, globals=globals(), number=10),
    "seconds.")

print("Using list comprehension takes: ",
    timeit.timeit('search_d(haystack, needle)', setup, globals=globals(), number=10),
    "seconds.")

print("Both functions together take: ",
    timeit.timeit('[func(haystack, needle) for func in (search_c, search_d)]',
    setup, globals=globals(), number=10), "seconds.")

You can run these examples a few times.  What happens if you are regenerating your random data every time?  You could also try changing the data size, to see what effect that has.

In [ ]:
# Generate some sample data again
setup = 'haystack = random.sample(range(1,1000), 512); needle = random.choice(haystack)'

In [ ]:
# Run the same example again
print("Using a generator expression takes: ",
    timeit.timeit('search_c(haystack, needle)', setup, globals=globals(), number=10),
    "seconds.")

print("Using list comprehension takes: ",
    timeit.timeit('search_d(haystack, needle)', setup, globals=globals(), number=10),
    "seconds.")

print("Both functions together take: ",
    timeit.timeit('[func(haystack, needle) for func in (search_c, search_d)]',
    setup, globals=globals(), number=10), "seconds.")

In [ ]:
# Generate some sample data (third time)
setup = 'haystack = random.sample(range(1,1000), 512); needle = random.choice(haystack)'

In [ ]:
# And again
print("Using a generator expression takes: ",
    timeit.timeit('search_c(haystack, needle)', setup, globals=globals(), number=10),
    "seconds.")

print("Using list comprehension takes: ",
    timeit.timeit('search_d(haystack, needle)', setup, globals=globals(), number=10),
    "seconds.")

print("Both functions together take: ",
    timeit.timeit('[func(haystack, needle) for func in (search_c, search_d)]',
    setup, globals=globals(), number=10), "seconds.")

Look carefully at your answers and think about what they mean.  What impact is our use of random data having here?

Importantly, can you now say anything about what the difference between using a generator expression and a list comprehension is?

Okay, so I confess that this was a trick question.  There is no answer to "which one is faster?" without knowing the context within which they will be used.  A better question would be "which one is better for my use case?"

Context is very important.  Our use of random data helped to confuse things, unless you thought carefully about the implications.  If you regenerated the data each time, you will have seen the execution times increase and decrease at random.  This is because the location of `needle` in the `haystack` list will have changed each time.  You should therefore have been able to guess from the time taken to find it whether the needle was nearer to the beginning, or the end of the list.

The generator expression is better for large problems where the dataset is used only once, then discarded.  This is because it is _lazy_.  In computer science, this term applied to this use case means that the data is not generated until it is needed.  This may save on start up costs and memory usage.  Generators are very powerful.  You can read more about them [here](https://wiki.python.org/moin/Generators).

In contrast, the list comprehension will be much faster for small problems that fit entirely in memory.  They load all of the data into memory at the start.  This may also be an advantage where you may need to search within the same list many times over, provided that it fits in memory.

<div class="alert alert-block alert-info">
    <b>Note:</b> You can use `/usr/bin/time` or the bash builtin `time` to time whole programs, but that does not usually scale very well with large, long running programs and will not tell you where time was spent.

[timeit](https://docs.python.org/3/library/timeit.html) has a lot of other useful methods of calling its funcitons, including decorators and a commandline interface, so it is worth reading through the documentation when you are thinking about timing something.
</di

### cProfile
Sometimes you need finer grained timing information than [timeit](https://docs.python.org/3/library/timeit.html) can provide.  Fortunately, Python also provides [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile) for this purpose.  Let's see what information this can give us.

The steps to using cProfile are simple:
* Import the [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile) module
* Call `cProfile.run()` or `cProfile.runctx()` with the function you want to profile

Let's see what it can tell us about our simple example above.  First, we'll import it:

In [ ]:
import cProfile

Now we can run our two functions with `cProfile.runctx()` to run them within a particular context.  This is useful when you may want to use different data sets.

In [ ]:
haystack = random.sample(range(1,2000), 600)
needle = random.choice(haystack)

print("Running search_c")
cProfile.runctx('search_c(haystack, needle)', globals(), locals())

print("Running search_d")
cProfile.runctx('search_d(haystack, needle)', globals(), locals())

Note execution time is listed as 0.00 for both!  This is because of the _deterministic profiling_ approach used by cprofile.  Read the documentation for details on what this means.  The moral here is that if you want accurate timing information, use [timeit](https://docs.python.org/3/library/timeit.html) instead.  It is important to understand not only the information you get from these tools, but also what interesting "features" they have.

When determinstic profiling is appropriate, we would get a much more detailed breakdown of where time is being spent.  However, you must use [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile) carefully.  Profiling an entire, large project is often not possible unless you are able to use a LOT of system resources to do it, and it will be very, very slow.

[cProfile](https://docs.python.org/3/library/profile.html#module-cProfile) is more powerful than this simple example; see the documentation for other useful examples, such as outputting to a file.  This is a particularly useful feature when working with applications running under a batch system or workload manager such as Slurm or PBS.

### Tracemalloc

This is another tool that you should be able to make use of in your Python projects.  Like [timeit](https://docs.python.org/3/library/timeit.html) and [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile), it is also part of the standard Python 3 library and should be installed anywhere that the Python 3 interpreter is available.

Again, usage is fairly straightforward:

* Import the tracemalloc module
* Start the trace
* Take snapshots at intervals
* Process the snapshots

Let's try it with our simple examples.

In [ ]:
import tracemalloc

haystack = random.sample(range(1,2000), 1024)
needle = random.choice(haystack)

In [ ]:
print("Running search_c")
tracemalloc.start()
search_c(haystack, needle)

snapshot = tracemalloc.take_snapshot()
tracemalloc.stop()

top_stats = snapshot.statistics('lineno')

print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)


In [ ]:
print("Running search_d")
tracemalloc.start()
search_d(haystack, needle)

snapshot = tracemalloc.take_snapshot()
tracemalloc.stop()

top_stats = snapshot.statistics('lineno')

print("[ Top 10 ]")
for stat in top_stats[:10]:
   print(stat)


This way of presenting the captured traces is an example from the documentation, which lists the 10 most memory hungry lines of code.  You don’t get a listing of a full 10 here as the program is too short, but this gives us a feel for what tracemalloc can do.  Each line is listed in memory usage order with a summary of how much memory it uses at the end.

What do you notice about the output from these two examples?  How do they compare?

As with [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile), it would be impractical to use this for tracing memory usage of an entire project so it must be used with care.  See the [tracemalloc](https://docs.python.org/3/library/tracemalloc.html) documentation for more examples, and other ways of using it.

## What have we learned?

Now that you've used [timeit](https://docs.python.org/3/library/timeit.html), [cProfile](https://docs.python.org/3/library/profile.html#module-cProfile) and [tracemalloc](https://docs.python.org/3/library/tracemalloc.html) to examine these two, what can you say about how they work?

The question about which one was faster is clearly too simplistic.

The first one, `search_c`, uses a _generator expression_, which lazily generates the values as you ask for them.  Read more about this in PEP 289 if you are interested.  They were proposed in ~2002 to combat the problem of high memory use of list comprehensions.

The second one, `search_d`, uses a _list comprehension_, generating the entire list in one line.  This is held in memory.  List comprehension has been present in the Python language standard since the beginning, so you will see a lot of examples using it.

The first one uses less memory, due to its laziness, but results in a lot of function calls.  The second one is faster for lists that fit in memory, especially if the list will be re-used, with only ~6 function calls.

So, if memory is limited (large data set), lazy expressions are better.

On the other hand, if you have plenty of memory and you want to make heavy use of the list, you may find it more beneficial to use a list comprehension.

Remember, speed of execution on a small test dataset can be misleading!

# What about the hardware

We've touched on how to improve our Python, but what about how to make better use of the hardware itself?

We want to consider how your application uses the computer hardware

This is much more difficult in Python!  The high level of abstraction means we can’t be sure about data layout, and that means we can’t ensure efficient L1/L2/L3 cache usage in the processor, or realistically influence prefetching in any useful way.

Garbage collection leads to memory fragmentation, and can make debugging harder too, as the true errors may be hidden and some errors transformed beyond recognition.  For example, invalid reference errors can turn into logic errors, and appear much later in the program than they actually occur.

Abstraction also means that *native Python cannot vectorise*.  So, we now we find our code can’t be vectorised, and you’ve already told us not to use threading!

So what do we do?  This sounds awful!  Even with all the care and attention we’ve given to our hotspots, this will be slower than using C/C++ or Fortran, and we’re limited to a single core!

Given that multicore chips are everywhere, increasing in core count and decreasing in frequency, we need a way around this.

## Numpy

[NumPy](https://numpy.org) is a well established, highly respected third-party Python package for numerical processing.

* Intended for scientific computing
* Highly optimised routines are actually written in C, with a nice Python interface
    * Vectorisation is now possible (over NumPy arrays)!

So, let's compare [NumPy](https://numpy.org) with our generator expressions and list comprehensions using the tools we've just learned about!  For this, we'll need to import NumPy and  define a suitable function using it.

In [ ]:
import numpy as np

Some test functions, showing various ways of doing the same thing.

In [ ]:
def forloop(size):
    array = [0]*size

    for i in range(size):
        array[i] = float(i)**2

    return sum(array)

def generatorexpr(size):
    array = [float(i)**2 for i in range(size)]
    return sum(array)

def listcomprehension(size):
    array = (float(i)**2 for i in range(size))
    return sum(array)

def usingnumpy(size):
    total = np.sum(np.arange(0, size)**2, dtype='d')
    return total

Now let's compare them!

In [ ]:
setup ='N = 10000'

print("Using a for loop takes ",
    timeit.timeit('[func(N) for func in (forloop,)]', setup, globals=globals(), number=10),
    "seconds.")

print("Using a generator expression takes ",
    timeit.timeit('[func(N) for func in (generatorexpr,)]', setup, globals=globals(), number=10),
    "seconds.")

print("Using a list comprehension takes ",
    timeit.timeit('[func(N) for func in (listcomprehension,)]', setup, globals=globals(), number=10),
    "seconds.")
    
print("Using NumPy takes ",
    timeit.timeit('[func(N) for func in (usingnumpy,)]', setup, globals=globals(), number=10),
    "seconds.")


Interesting!  Look how much faster NumPy is than the others, even on this small example.

Now let's see what cProfile makes of it!

In [ ]:
print("Using a for loop: ")
cProfile.run('forloop(10000)')

print("Using a generator expression: ")
cProfile.run('generatorexpr(10000)')

print("Using a list comprehension: ")
cProfile.run('listcomprehension(10000)')

print("Using Numpy: ")
cProfile.run('usingnumpy(10000)')

Interesting, isn't it?  Despite the larger number of function calls, NumPy is still the fastest.  It also uses memory more efficiently than the other examples.

The moral here is to use [NumPy](https://numpy.org), not roll your own, if you are working with numerical data.

(Don’t forget to look at SciPy too.  I haven’t mentioned it as we haven’t got time in this tutorial.)

# Multiprocessing

So far this has all been boring, single process code.

This is 2020.  Some of us have had multiprocessor systems since at least the mid-1980s!  Fortunately for us, python 3 added a new module to the library: [multiprocessing](https://docs.python.org/3/library/multiprocessing.html)  This is now essential for any budding data scientist or most normal people who want to do more with Python than just write the odd script.

[multiprocessing](https://docs.python.org/3/library/multiprocessing.html) allows you to use all the cores _without_ using the threading module.  Excellent!
* Bypasses GIL-related problems
* Similar API to threading

* Adds additional features, such as thread pools
    * Great for data parallelism, which is exactly what we want

If you are already familiar with OpenMP, you will find some of the ways of using [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) straightforward, but it also has many more features familiar to anyone who is well versed in the theory behind parallel programming.  Many of these have analogues in other languages.

Let's start with a very boring but familiar example:

In [ ]:
import os
from multiprocessing import Process

def greet():
    print('Hello from process ', os.getpid(), ' spawned by ', os.getppid())


p = Process(target=greet, args=())
p.start()
p.join()

This is boring, as it only spawns one process.  At its most basic, processes are spawned by `Process()`.  The `target` argument gives the function to execute, and `args` supplies a list of arguments.

The PID and PPID are supplied by the OS.

<div class="alert alert-block alert-warning">
<b>Warning:</b> A forked process using the standard libraries might use 10-20MB of RAM.  That doesn’t sound like a lot, but it adds up.  If you are using a lot of libraries and a lot of data, you could end up using a lot more RAM, and could end up swapping!!
</div>

### Process Pools

These are an easier way to get started, and our last example for this very powerful module.

All you do is create a pool, or group of processes, and submit tasks to it via the Pool class:

`Pool.apply(func[, args[, kwds]])`
    * Only executes func on one worker in the pool
    * Call func with arguments args and keyword arguments kwds
    * Blocking

* `Pool.map(func, iterable[, chunksize])`
    * Parallel equivalent to map() built-in function
    * Blocking
    * Long iterables cause high memory usage:  Consider imap() or imap_unordered() with a chunksize instead


`Pool.map` is a parallel equivalent of the `map()` built-in function (it supports only one iterable argument though, for multiple iterables see `starmap()`). It blocks until the result is ready.

This method chops the iterable into a number of chunks which it submits to the process pool as separate tasks. The (approximate) size of these chunks can be specified by setting chunksize to a positive integer.   Note that it may cause high memory usage for very long iterables. Consider using `imap()` or `imap_unordered()` with explicit chunksize option for better efficiency.

There are also _asynchronous_ equivalents equivalents of these Pool functions: `map_async()` and `apply_async()`.

How about another example?

In [ ]:
import multiprocessing as mp

def greet(task):
    print("Hello from process ", os.getpid(), " spawned by ", os.getppid(), " and given task ", task)

with mp.Pool(4) as p:
    p.map(greet, [1,2,3,4])

This time we are using a pool of processes.

By default, the size of the pool is derived from `os.cpu_count()`

You must use the pool to manage the processes, or make sure you `close()` or `terminate()` them manually, or they will hang around after your program exits and this will make you unpopular.

Do not rely on the GC to clean up the mess for you.

Alright, that was still boring.  How about using multiprocessing *with* NumPy?

In [ ]:
with mp.Pool() as p:
    # apply_async returns immediately, providing us with a proxy object
    results = [p.apply_async(np.sqrt, (x,)) for x in range(100)]

    # This accesses the proxy for results; get() waits
    # for task completion before returning the results
    roots = [r.get() for r in results]
    print(roots)

## Other Pool functions you should know:

* `Pool.close()`
    * Do not allow the Pool to accept any more work

* `Pool.terminate()`
    * Stop all work immediately on all workers

* `Pool.join()`
    * Wait for worker processes to exit
    * Must only be called after `close()` or `terminate()`


The [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) module is extremely powerful, but unforunately we don't have time to go through all of the features offers here so do take the time to read through the documentation and play around with it yourself.

There are also barriers, semaphores and many structures that may be useful.  You can share state, and many other things.

Read the documentation and familiarise yourself with the [multiprocessing programming guidelines](https://docs.python.org/3/library/multiprocessing.html#multiprocessing-programming)!  They will help you to avoid common slip-ups and maintain a good coding style.

A word of warning though:

There are frameworks built on multiprocessing.  None of the ones I have seen so far are any good.  Most re-invent collective communication, rather than using the collectives in MPI, and they attempt to optimise it.  This assumes that your hardware will not optimise it for you.  If the hardware does optimise it (which in any real supercomputer, it will do) and you use these frameworks, then you lose out.

# MPI4Py

This is not very Pythonic, because it is built on the C API for MPI, the message passing interface.  This can introduce problems with data type conversion.  However, it is worth knowing if you plan on working on supercomputers using Python.  If you are familiar with using MPI with Fortran or C already, then you will not find it difficult to make the leap to using it in Python.

[MPI4Py](https://mpi4py.readthedocs.io/en/stable/) _must_ be built against the MPI on the supercomputer you are using because it relies on hardware-specific libraries.

If you are on a HPE Cray supercomputer, it will have been built for you and optimised.  Look for the `cray-python` modules and use those for your jobs.

On other systems, you may need to build it yourself:

	module load <mpi of choice>
	python -m pip install --user mpi4py

However, it is better if the local sysadmins will install it for you (and support it).  That way you can be sure that you are using the correct MPI libraries.

If there are multiple versions of Python (and compilers) available, make sure you have the right modules loaded first.

Consider where you want to install it.  By default this will go into your home directory, but you may not have sufficient space.  You may also want to use a venv or similar.

In [ ]:
from mpi4py import MPI

size = MPI.COMM_WORLD.Get_size()
rank = MPI.COMM_WORLD.Get_rank()
name = MPI.Get_processor_name()

msg = "Hello, World! from process {0} of {1} on {2}"
print(msg.format(rank, size, name))

Function names are familiar MPI functions.

There is also support for MPI-IO, which is brilliant for processing large files on parallel file systems.

You do not need to call MPI_Init or MPI_Finalize in Python.  This is done for you.

Load the modules you need and execute srun, mpirun or mpiexec as normal.

The [MPI4Py tutorial](https://mpi4py.readthedocs.io/en/stable/tutorial.html) has many simple examples.

# All three combined!

What does it look like if we use multiprocessing, NumPy and MPI4Py all together?  Here's a simple and somewhat contrived example:

In [ ]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    # This is our root process
    sendbuf = np.arange(100, dtype='i')
else:
    sendbuf = np.empty(100, dtype='i')

comm.Bcast(sendbuf, root=0)

with mp.Pool(4) as p:
    # apply_async returns immediately, providing us with a proxy object
    results = [p.apply_async(np.sqrt, (x,)) for x in sendbuf]

    # This accesses the proxy for results; get() waits
    # for task completion before returning the results
    roots = [r.get() for r in results]
    print(roots)

    p.close()

if rank == 0:
    recvbuf = np.empty([size, 100], dtype='i')
    comm.Gather(sendbuf, recvbuf, root=0)

    if rank == 0:
        print(recvbuf)


Be careful what you pass to the scheduler, as you do not want to oversubscribe the compute nodes.  That would hurt performance and you could run out of memory.

If you want to get fancy, you can do checks at start up to ensure that only one MPI rank has been spawned per node.

Watch out for NumPy compiled against a parallel BLAS!

Note that shared data structures should be used as little as possible.  Shared data structures really only arrived in Python 3.8, which is another reason why I didn’t include them.  Python 3.8 has many improvements over its predecessors, and this is one of them.  In future, this problem will go away but do consider it when you need backwards compatibility.

NumPy uses the BLAS for many of its operations.  The BLAS is just the API specification, and the implementation can vary from the slowest performing reference implementation to fast, multi-threaded versions like Intel MKL, libSci (Cray), ATLAS and OpenBLAS.  If it is multi-threaded, then you may not want to use multiple threads too.  This would oversubscribe your nodes.

# Time to wrap up
You can now move to the conclusion of this workshop !

* [Conclusion](2-WKSHP-Conclusion.ipynb)